# Live Trading Modular Reference NotebookThis notebook documents the modular building blocks that make up the live Alpaca trading bot. It is designed to be used **during live market hours** so you can inspect and adjust configuration without stopping the bot. None of the cells below perform paper-trading simulation—they connect to the real-time broker objects configured in `config.py`.> ⚠️ **Warning:** Executing order-related cells while authenticated with live credentials will place orders in the live market. Double-check all parameters before running any trading commands.

## 1. Environment prerequisites1. Export your Alpaca API credentials so that the modules can authenticate against the live endpoint:   ```bash   export ALPACA_K="your-key-id"   export ALPACA_SK="your-secret-key"   export ALPACA_PAPER="false"  # ensure live trading   ```2. (Optional) Review `requirements.txt` and install the dependencies if this is a fresh environment.3. Run the cells below to inspect and orchestrate the modular components.

In [ ]:
from __future__ import annotationsimport asyncioimport inspectfrom dataclasses import asdict, fieldsimport nest_asynciofrom bootstrap import ensure_requirementsfrom broker import AlpacaBrokerfrom config import AppConfig, AlpacaCredentials, StrategyConfigfrom indicators import IndicatorSetfrom risk import RiskManagerfrom strategy import EmaSmaStrategy# Guarantee compatibility with already-running event loops (common in notebooks)nest_asyncio.apply()# Safety: no automatic installation when running during market hours.ensure_requirements(auto_install=False)

## 2. Strategy configuration anatomyThe trading bot is configured via the `StrategyConfig` dataclass. Inspecting its attributes helps during live trading sessions when you need to confirm symbol routing, allocation limits, or streaming behaviour.

In [ ]:
strategy_config = StrategyConfig()for field in fields(strategy_config):    value = getattr(strategy_config, field.name)    print(f"{field.name}: {value}")

Use the cell below to override defaults—for example to switch to a different bullish/bearish pair or adjust the cash allocation fraction—**before** instantiating the rest of the system.

In [ ]:
custom_strategy = StrategyConfig(    bullish_symbol="SOXL",    bearish_symbol="SOXS",    allocation=strategy_config.allocation,    indicators=strategy_config.indicators,    risk=strategy_config.risk,    stream=strategy_config.stream,)custom_strategy.allocation.cash_fraction = 0.75  # allocate 75% of buying powercustom_strategy.stream.symbols = ["SOXL", "SOXS"]custom_strategy

## 3. Credential handling and broker facade`AlpacaBroker` wraps both the trading REST client and the live market data stream. The snippet below shows how credentials are supplied and which helper methods the broker exposes for trading operations.

In [ ]:
credentials = AlpacaCredentials.from_env()app_config = AppConfig(credentials=credentials, strategy=custom_strategy)broker = AlpacaBroker(app_config.credentials, app_config.strategy)# Summarize the key callable broker methods for quick reference.broker_operations = {    name: obj    for name, obj in inspect.getmembers(broker, predicate=callable)    if not name.startswith("_")}list(broker_operations.keys())

The following helper prints docstrings and signatures for the most important order- and stream-related methods. Review these before sending live orders.

In [ ]:
for name in [    "get_account",    "get_positions",    "submit_market_order",    "close_position",    "close_all_positions",    "subscribe_bars",    "run_stream",]:    func = broker_operations[name]    signature = str(inspect.signature(func))    doc = inspect.getdoc(func) or "No docstring."    print(f"{name}{signature}  {doc}")

## 4. Indicators module snapshot`IndicatorSet` keeps EMA, SMA, and ATR indicators synchronized with the latest bars. Use it to monitor trend signals during the session.

In [ ]:
indicators = IndicatorSet.from_config(    ema_period=app_config.strategy.indicators.ema_period,    sma_period=app_config.strategy.indicators.sma_period,    atr_period=app_config.strategy.indicators.atr_period,)seed_bars = broker.get_seed_bars(    app_config.strategy.bullish_symbol,    app_config.strategy.indicators.seed_bars,)indicators.seed_from_bars(seed_bars)print("Latest EMA:", indicators.latest_ema)print("Latest SMA:", indicators.latest_sma)

## 5. Risk manager overviewThe `RiskManager` automates guardrails such as flattening positions near the close, ensuring buying power availability, and hedging logic. Inspect its callable surface along with configuration values while markets are open.

In [ ]:
risk_manager = RiskManager(broker, app_config.strategy.risk)risk_summary = asdict(app_config.strategy.risk)print("Risk configuration:")for key, value in risk_summary.items():    print(f"  {key}: {value}")# Display coroutine signatures for situational awareness.for name, func in inspect.getmembers(risk_manager, predicate=callable):    if name.startswith("_"):        continue    signature = str(inspect.signature(func))    print(f"RiskManager.{name}{signature}")

## 6. Strategy orchestration hooks`EmaSmaStrategy` contains the crossover logic used during live trading. You can inject diagnostic hooks (logging, notifications) or trigger manual calls to `on_bar` while inspecting incoming bars.

In [ ]:
strategy = EmaSmaStrategy(broker, indicators, risk_manager)print("Strategy is monitoring:", app_config.strategy.all_symbols())print("Attributes:", [name for name in strategy.__dict__.keys() if not name.startswith("_")])print("Public coroutine methods:")for name, func in inspect.getmembers(strategy, predicate=callable):    if name.startswith("_"):        continue    print(" -", name, inspect.signature(func))

## 7. Live stream wiring (manual start)The final cell shows how to start the live data stream. Execute it only when you are ready to process real-time bars. The `handle_bar` coroutine delegates to `strategy.on_bar`, so your trading logic runs automatically. Stop the stream with `await broker.stop_stream()` if you need to pause trading.

In [ ]:
async def handle_bar(bar):    await strategy.on_bar(bar)broker.subscribe_bars(handle_bar, *app_config.strategy.all_symbols())# Uncomment the line below to start streaming live market data.# await broker.run_stream()